<a href="https://colab.research.google.com/github/aydinmyilmaz/Spark-NLP/blob/master/NER_Training_with_custom_word_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import os

# Install java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version

# Install pyspark
! pip install --ignore-installed -q pyspark==2.4.4
! pip install --ignore-installed -q spark-nlp==2.5.4

openjdk version "1.8.0_265"
OpenJDK Runtime Environment (build 1.8.0_265-8u265-b01-0ubuntu2~18.04-b01)
OpenJDK 64-Bit Server VM (build 25.265-b01, mixed mode)
     |████████████████████████████████| 215.7MB 65kB/s 
     |████████████████████████████████| 204kB 55.0MB/s 
     |████████████████████████████████| 133kB 2.6MB/s 


In [2]:
import sparknlp

spark = sparknlp.start() # for GPU training >> sparknlp.start(gpu = True) # for Spark 2.3 =>> sparknlp.start(spark23 = True)

from sparknlp.base import *
from sparknlp.annotator import *

print("Spark NLP version", sparknlp.version())

print("Apache Spark version:", spark.version)

spark

Spark NLP version 2.5.4
Apache Spark version: 2.4.4


# 1st CoNLL File with all NERs

In [3]:
with open("/content/project_german_med_api_0906.conll") as f:
    train_txt =f.read()

print (train_txt[:500])

-DOCSTART- -X- -X- O

Thread -X- -X- O
: -X- -X- O
[Nierenkolik -X- -X- O
ohne -X- -X- O
Stein -X- -X- B-MEDICAL_CONDITION
und -X- -X- O
ohne -X- -X- O
Gries?] -X- -X- O
Text -X- -X- O
: -X- -X- O
[Eine -X- -X- O
Nierenbeckenstenose -X- -X- B-DIAGLAB_PROCEDURE
würde -X- -X- O
sicherlich -X- -X- O
, -X- -X- O
wenn -X- -X- O
sie -X- -X- O
vorhanden -X- -X- O
wäre -X- -X- O
, -X- -X- O
bemerkt -X- -X- O
worden -X- -X- O
im -X- -X- O
Krankenhaus -X- -X- O
. -X- -X- O

] -X- -X- O

Thread -X- -X- O
:


In [35]:
import pandas as pd
pdDF = pd.read_csv('/content/Ger_Med_CoNLL_changed.conll', sep=' ', header=0, index_col=None) #project_german_med_api_0906.conll
pdDF.head()

,-DOCSTART-,-X-,-X-.1,O
0,Thread,-X-,-X-,O
1,:,-X-,-X-,O
2,[Nierenkolik,-X-,-X-,O
3,ohne,-X-,-X-,O
4,Stein,-X-,-X-,B-MEDICAL_CONDITION


In [21]:
pdDF.O.value_counts()

O                          328279
B-MEDICAL_CONDITION         13061
I-TIME_INFORMATION           6482
B-BODY_PART                  4875
B-TIME_INFORMATION           4753
B-MEASUREMENT                4205
B-TREATMENT                  3811
B-PERSON                     3795
B-MEDICATION                 3248
B-STATE_OF_HEALTH            2710
B-PROCESS                    2035
B-DIAGLAB_PROCEDURE          1781
I-MEASUREMENT                1726
B-BIOLOGICAL_CHEMISTRY       1483
B-BODY_FLUID                 1260
B-BIOLOGICAL_PARAMETER        943
I-MEDICAL_CONDITION           692
B-LOCAL_SPECIFICATION         570
B-MEDICAL_DEVICE              550
I-DOSING                      429
I-STATE_OF_HEALTH             388
B-DOSING                      386
I-MEDICATION                  297
B-MEDICAL_SPECIFICATION       285
I-PERSON                      266
I-TREATMENT                   243
I-LOCAL_SPECIFICATION          67
I-BIOLOGICAL_CHEMISTRY         53
I-BIOLOGICAL_PARAMETER         48
I-MEDICAL_DEVI

# 2nd CoNLL File B-TISSUE, I-PROCESS  removed

In [3]:
from sparknlp.training import CoNLL

conll_data = CoNLL().readDataset(spark, '/content/project_german_med_api_0906.conll')

conll_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Thread : [Nierenk...|[[document, 0, 15...|[[document, 0, 15...|[[token, 0, 5, Th...|[[pos, 0, 5, -X-,...|[[named_entity, 0...|
|                   ]|[[document, 0, 0,...|[[document, 0, 0,...|[[token, 0, 0, ],...|[[pos, 0, 0, -X-,...|[[named_entity, 0...|
|Thread : [Wandern...|[[document, 0, 24...|[[document, 0, 24...|[[token, 0, 5, Th...|[[pos, 0, 5, -X-,...|[[named_entity, 0...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 3 rows



In [5]:
from pyspark.sql import functions as F
conll_data.select(F.explode(F.arrays_zip('token.result','label.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("ground_truth")).groupBy('ground_truth').count().orderBy('count', ascending=False).show(100,truncate=False)


+-----------------------+------+
|ground_truth           |count |
+-----------------------+------+
|O                      |328313|
|B-MEDICAL_CONDITION    |13061 |
|I-TIME_INFORMATION     |6482  |
|B-BODY_PART            |4875  |
|B-TIME_INFORMATION     |4753  |
|B-MEASUREMENT          |4205  |
|B-TREATMENT            |3811  |
|B-PERSON               |3795  |
|B-MEDICATION           |3248  |
|B-STATE_OF_HEALTH      |2710  |
|B-PROCESS              |2035  |
|B-DIAGLAB_PROCEDURE    |1781  |
|I-MEASUREMENT          |1726  |
|B-BIOLOGICAL_CHEMISTRY |1483  |
|B-BODY_FLUID           |1260  |
|B-BIOLOGICAL_PARAMETER |943   |
|I-MEDICAL_CONDITION    |692   |
|B-LOCAL_SPECIFICATION  |570   |
|B-MEDICAL_DEVICE       |550   |
|I-DOSING               |429   |
|I-STATE_OF_HEALTH      |388   |
|B-DOSING               |386   |
|I-MEDICATION           |297   |
|B-MEDICAL_SPECIFICATION|285   |
|I-PERSON               |266   |
|I-TREATMENT            |243   |
|I-LOCAL_SPECIFICATION  |67    |
|I-BIOLOGI

In [6]:
! wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.de.300.vec.gz
!gunzip cc.de.300.vec.gz

--2020-09-14 10:11:40--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.de.300.vec.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.75.142, 104.22.74.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.75.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1278030050 (1.2G) [binary/octet-stream]
Saving to: ‘cc.de.300.vec.gz’

cc.de.300.vec.gz    100%[===================>]   1.19G  14.0MB/s    in 88s     

2020-09-14 10:13:09 (13.8 MB/s) - ‘cc.de.300.vec.gz’ saved [1278030050/1278030050]



# Build Pipeline

In [7]:
# Split data set
from sparknlp.training import CoNLL

#conll_data = CoNLL().readDataset(spark, './eng.train')

(training_data, test_data) = conll_data.randomSplit([0.7, 0.3], seed = 100)

print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 20450
Test Dataset Count: 8661


In [26]:
'''
word_embeddings = WordEmbeddings()\
  .setInputCols(["document", "token"])\
  .setOutputCol("embeddings")\
  .setStoragePath('cc.de.300.vec', "TEXT")\
  .setDimension(300)
  '''

custom_embeddings = WordEmbeddings()\
  .setInputCols(["document", "token"])\
  .setOutputCol("word_embeddings")\
  .setStoragePath('model_w2v.bin', "BINARY")\
  .setDimension(300)


In [27]:
!mkdir ner_logs

mkdir: cannot create directory ‘ner_logs’: File exists


In [29]:
nerTagger = NerDLApproach()\
  .setInputCols(["sentence", "token", "embeddings"])\
  .setLabelColumn("label")\
  .setOutputCol("ner")\
  .setMaxEpochs(2)\
  .setLr(0.003)\
  .setPo(0.005)\
  .setBatchSize(8)\
  .setRandomSeed(0)\
  .setVerbose(1)\
  .setValidationSplit(0.2)\
  .setEvaluationLogExtended(True) \
  .setEnableOutputLogs(True)\
  .setIncludeConfidence(True)\
  .setOutputLogsPath('ner_logs') # if not set, logs will be written to ~/annotator_logs

ner_pipeline = Pipeline(stages=[
          word_embeddings,
          nerTagger
 ])

## Fitting

In [30]:
%%time

ner_model = ner_pipeline.fit(training_data)

CPU times: user 347 ms, sys: 50 ms, total: 397 ms
Wall time: 27min 46s


In [31]:
#! cd ~/annotator_logs && ls -lt
!cd ner_logs && ls -lt

total 28
-rw-r--r-- 1 root root 4685 Sep 14 12:05 NerDLApproach_13d42060b86d.log
-rw-r--r-- 1 root root 9362 Sep 14 11:57 NerDLApproach_daf954c7e368.log
-rw-r--r-- 1 root root 4618 Sep 14 10:30 NerDLApproach_c44e43c7d6b1.log


In [33]:
!head -n 45 ner_logs/NerDLApproach_13d42060b86d.log

Name of the selected graph: ner-dl/blstm_38_300_128_200.pb
Training started - total epochs: 2 - lr: 0.003 - batch size: 8 - labels: 33 - chars: 96 - training examples: 16360


Epoch 1/2 started, lr: 0.003, dataset size: 16360


Epoch 1/2 - 748.34s - loss: 8468.353 - batches: 2047
Quality on validation dataset (20.0%), validation examples = 4090
time to finish evaluation: 54.94s
label	 tp	 fp	 fn	 prec	 rec	 f1
B-PERSON	 352	 29	 146	 0.9238845	 0.7068273	 0.80091006
B-TIME_INFORMATION	 525	 142	 141	 0.78710645	 0.7882883	 0.78769696
B-MEDICAL_SPECIFICATION	 7	 1	 32	 0.875	 0.17948718	 0.29787233
B-BODY_FLUID	 144	 12	 19	 0.9230769	 0.8834356	 0.9028213
I-TREATMENT	 0	 2	 26	 0.0	 0.0	 0.0
B-STATE_OF_HEALTH	 314	 64	 83	 0.8306878	 0.790932	 0.8103226
I-STATE_OF_HEALTH	 27	 9	 34	 0.75	 0.44262296	 0.55670106
B-BIOLOGICAL_CHEMISTRY	 170	 55	 71	 0.75555557	 0.7053942	 0.7296138
B-DOSING	 18	 1	 33	 0.94736844	 0.3529412	 0.5142857
I-TIME_INFORMATION	 752	 95	 205	 0.88783944	 0.78578

# Test Data- Transform

In [ ]:
word_embeddings.transform(test_data).write.parquet('ner_dl_test.parquet')

In [34]:
from sparknlp.training import CoNLL

#test_data = CoNLL().readDataset(spark, './eng.testa')

test_data = word_embeddings.fit(test_data).transform(test_data)

test_data.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|          embeddings|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                %-|]|[[document, 0, 3,...|[[document, 0, 3,...|[[token, 0, 3, %-...|[[pos, 0, 3, -X-,...|[[named_entity, 0...|[[word_embeddings...|
|' Sind Sie sicher...|[[document, 0, 39...|[[document, 0, 39...|[[token, 0, 0, ',...|[[pos, 0, 0, -X-,...|[[named_entity, 0...|[[word_embeddings...|
|( Aber du ist bes...|[[document, 0, 15...|[[document, 0, 15...|[[token, 0, 0, (,...|[[pos, 0, 0, -X-,...|[[named_entity, 0...|[[word_embeddings...|
+--------------------+--------------------+--------------------+--------------------+--------------------+

In [35]:
predictions = ner_model.transform(test_data)
predictions.show(3)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|          embeddings|                 ner|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                %-|]|[[document, 0, 3,...|[[document, 0, 3,...|[[token, 0, 3, %-...|[[pos, 0, 3, -X-,...|[[named_entity, 0...|[[word_embeddings...|[[named_entity, 0...|
|' Sind Sie sicher...|[[document, 0, 39...|[[document, 0, 39...|[[token, 0, 0, ',...|[[pos, 0, 0, -X-,...|[[named_entity, 0...|[[word_embeddings...|[[named_entity, 0...|
|( Aber du ist bes...|[[document, 0, 15...|[[document, 0, 15...|[[token, 0, 0, (,...|[[pos, 0, 0, -X-,...|[[named_entity, 0...|[[word_embeddings...|[[

# Test set Evaluation

In [36]:
import pyspark.sql.functions as F

predictions.select(F.explode(F.arrays_zip('token.result','label.result','ner.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("ground_truth"),
        F.expr("cols['2']").alias("prediction")).show(truncate=False)

+-----------+------------+----------+
|token      |ground_truth|prediction|
+-----------+------------+----------+
|%-|]       |O           |O         |
|'          |O           |O         |
|Sind       |O           |O         |
|Sie        |O           |O         |
|sicher     |O           |O         |
|,          |O           |O         |
|mit        |O           |O         |
|einem      |O           |O         |
|behinderten|O           |O         |
|Kind       |B-PERSON    |B-PERSON  |
|klar       |O           |O         |
|zu         |O           |O         |
|kommen     |O           |O         |
|?          |O           |O         |
|Sie        |O           |O         |
|müssen     |O           |O         |
|auch       |O           |O         |
|an         |O           |O         |
|Ihre       |O           |O         |
|anderen    |O           |O         |
+-----------+------------+----------+
only showing top 20 rows



In [37]:
from sklearn.metrics import classification_report

preds_df = predictions.select(F.explode(F.arrays_zip('token.result','label.result','ner.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("ground_truth"),
        F.expr("cols['2']").alias("prediction")).toPandas()


print (classification_report(preds_df['ground_truth'], preds_df['prediction']))

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                         precision    recall  f1-score   support

 B-BIOLOGICAL_CHEMISTRY       0.75      0.72      0.73       395
 B-BIOLOGICAL_PARAMETER       0.76      0.47      0.58       284
           B-BODY_FLUID       0.91      0.88      0.89       372
            B-BODY_PART       0.91      0.85      0.88      1453
               B-DEGREE       0.00      0.00      0.00         1
    B-DIAGLAB_PROCEDURE       0.76      0.70      0.73       532
               B-DOSING       0.85      0.32      0.46       110
  B-LOCAL_SPECIFICATION       0.83      0.80      0.82       176
          B-MEASUREMENT       0.71      0.71      0.71      1245
    B-MEDICAL_CONDITION       0.84      0.80      0.82      3913
       B-MEDICAL_DEVICE       0.92      0.26      0.41       168
B-MEDICAL_SPECIFICATION       0.62      0.33      0.43        80
           B-MEDICATION       0.73      0.63      0.68       966
               B-PERSON       0.90      0.79      0.85      1163
              B-PROCESS 

# save trained model

In [19]:
ner_model.stages

[WORD_EMBEDDINGS_MODEL_6ea4c46e28d6, NerDLModel_a48125f8dbc4]

In [20]:
ner_model.stages[1].write().overwrite().save('NER_ccde300_20200914_de')

In [21]:

!ls -lt

total 4423340
drwxr-xr-x 4 root root       4096 Sep 14 11:05 NER_ccde300_20200914_de
drwxr-xr-x 2 root root       4096 Sep 14 10:21 ner_logs
-rw-r--r-- 1 root root    7012058 Sep 14 10:08 project_german_med_api_0906.conll
drwxr-xr-x 1 root root       4096 Aug 27 16:39 sample_data
-rw-r--r-- 1 root root 4522468726 Jan 18  2019 cc.de.300.vec


# Prediction Pipeline

In [ ]:
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

sentence = SentenceDetector()\
    .setInputCols(['document'])\
    .setOutputCol('sentence')

token = Tokenizer()\
    .setInputCols(['sentence'])\
    .setOutputCol('token')

word_embeddings = WordEmbeddings()\
  .setInputCols(["document", "token"])\
  .setOutputCol("embeddings")\
  .setStoragePath('cc.de.300.vec', "TEXT")\
  .setDimension(300)
    
loaded_ner_model = NerDLModel.load("NER_ccde300_20200914_de")\
 .setInputCols(["sentence", "token", "embeddings"])\
 .setOutputCol("ner")

converter = NerConverter()\
  .setInputCols(["document", "token", "ner"])\
  .setOutputCol("ner_span")

ner_prediction_pipeline = Pipeline(
    stages = [
        document,
        sentence,
        token,
        word_embeddings,
        loaded_ner_model,
        converter])

In [ ]:
empty_data = spark.createDataFrame([['']]).toDF("text")

prediction_model = ner_prediction_pipeline.fit(empty_data)

In [ ]:

preds = prediction_model.transform(sample_data)

preds.select(F.explode(F.arrays_zip("ner_span.result","ner_span.metadata")).alias("entities")) \
.select(F.expr("entities['0']").alias("chunk"),
        F.expr("entities['1'].entity").alias("entity")).show(truncate=False)

In [ ]:
from sparknlp.base import LightPipeline

light_model = LightPipeline(prediction_model)

In [ ]:
text = "Peter Parker is a nice guy and lives in New York."

result = light_model.annotate(text)

list(zip(result['token'], result['ner']))

In [ ]:
import pandas as pd

result = light_model.fullAnnotate(text)

ner_df= pd.DataFrame([(int(x.metadata['sentence']), x.result, x.begin, x.end, y.result) for x,y in zip(result[0]["token"], result[0]["ner"])], 
                      columns=['sent_id','token','start','end','ner'])
ner_df